In [20]:
import pandas as pd, os, re

BASE = '../data/raw/cbioportal/'
clin = pd.read_csv(os.path.join(BASE, 'clinical.tsv'), sep='\t')

print(clin.columns.tolist()[:30])       


['Study ID', 'Patient ID', 'Sample ID', 'Diagnosis Age', 'American Joint Committee on Cancer Metastasis Stage Code', 'Neoplasm Disease Lymph Node Stage American Joint Committee on Cancer Code', 'Neoplasm Disease Stage American Joint Committee on Cancer Code', 'American Joint Committee on Cancer Publication Version Type', 'American Joint Committee on Cancer Tumor Stage Code', 'Alcohol Consumption Frequency', 'Alcohol History Documented', 'Amplification Status', 'Angiolymphatic Invasion', 'Cancer Type', 'Cancer Type Detailed', 'Neoplasm American Joint Committee on Cancer Clinical Distant Metastasis M Stage', 'Neoplasm American Joint Committee on Cancer Clinical Regional Lymph Node N Stage', 'Neoplasm American Joint Committee on Cancer Clinical Primary Tumor T Stage', 'Neoplasm American Joint Committee on Cancer Clinical Group Stage', 'Daily Alcohol', 'Days to Sample Collection.', 'Last Alive Less Initial Pathologic Diagnosis Date Calculated Day Value', 'Days to Sample Procurement', 'Dise

In [21]:
[col for col in clin.columns if 'hpv' in col.lower()]

['Hpv status ish', 'Hpv status p16']

In [22]:
[col for col in clin.columns if 'stage' in col.lower()]

['American Joint Committee on Cancer Metastasis Stage Code',
 'Neoplasm Disease Lymph Node Stage American Joint Committee on Cancer Code',
 'Neoplasm Disease Stage American Joint Committee on Cancer Code',
 'American Joint Committee on Cancer Tumor Stage Code',
 'Neoplasm American Joint Committee on Cancer Clinical Distant Metastasis M Stage',
 'Neoplasm American Joint Committee on Cancer Clinical Regional Lymph Node N Stage',
 'Neoplasm American Joint Committee on Cancer Clinical Primary Tumor T Stage',
 'Neoplasm American Joint Committee on Cancer Clinical Group Stage',
 'Stage Other']

In [23]:
hpv_col   = 'Hpv status p16'
stage_col = 'Neoplasm Disease Stage American Joint Committee on Cancer Code'

mask = (
    clin[hpv_col].str.upper().eq('NEGATIVE') &
    clin[stage_col].str.contains(r'III|IV', flags=re.I, na=False)
)
subset_ids = clin.loc[mask, 'Sample ID'].tolist()
print(len(subset_ids))

53


In [24]:
import pandas as pd, os
BASE = '../data/raw/cbioportal/'

expr_head = pd.read_csv(os.path.join(BASE, 'mrna_z.txt'), sep='\t', nrows=3)
print(expr_head.columns[:5].tolist())

['STUDY_ID', 'SAMPLE_ID', 'TAP1', 'B2M', 'IRF1']


In [25]:
cna_head = pd.read_csv(os.path.join(BASE, 'cna.txt'), sep='\t', nrows=3)
print(cna_head.columns[:5].tolist())

['STUDY_ID', 'SAMPLE_ID', 'TAP1', 'B2M', 'IRF1']


In [26]:
# inspect mutation file header
mut_head = pd.read_csv(f'{BASE}mutations.txt', sep='\t', nrows=3)
print(mut_head.columns.tolist())

['STUDY_ID', 'SAMPLE_ID', 'TAP1', 'B2M', 'IRF1', 'STAT1', 'CXCL9', 'CXCL10', 'PSMB9', 'NLRC5', 'HLA-A', 'HLA-B']


In [27]:
BASE = '../data/raw/cbioportal/'

genes = ['TAP1','B2M','IRF1','STAT1','CXCL9',
         'CXCL10','PSMB9','NLRC5','HLA-A','HLA-B']

expr = (pd.read_csv(f'{BASE}mrna_z.txt', sep='\t')
          .set_index('SAMPLE_ID')
          .loc[subset_ids, genes]
          .astype(float)
          .T)

cna  = (pd.read_csv(f'{BASE}cna.txt', sep='\t')
          .set_index('SAMPLE_ID')
          .loc[subset_ids, genes]
          .astype(float)
          .T)

mut  = (pd.read_csv(f'{BASE}mutations.txt', sep='\t')
          .set_index('SAMPLE_ID')
          .loc[subset_ids, genes]
          .apply(pd.to_numeric, errors='coerce')
          .fillna(0)
          .T)

low_expr = expr <= -1.5
mut_pct  = (mut != 0).mean(axis=1)
del_pct  = (cna == -2).mean(axis=1)

score = (pd.DataFrame({'low_pct':     low_expr.mean(axis=1),
                       'mut_pct':     mut_pct,
                       'cnv_del_pct': del_pct})
           .assign(rescue_score=lambda d: d.low_pct * (1 - d.mut_pct - d.cnv_del_pct))
           .sort_values('rescue_score', ascending=False))

out_path = '../data/processed/initial_rescue_scores.csv'
os.makedirs('../data/processed', exist_ok=True)
score.to_csv(out_path)
print(out_path)

../data/processed/initial_rescue_scores.csv
